In [1]:
import numpy as np
import pandas as pd
import warnings
from datetime import datetime, timedelta
from matplotlib import pyplot as plt


warnings.filterwarnings('ignore')

For the baseline model, we calculate the reward function for each telescope on each day using $$f(D, R) = -\frac{1}{T(R)}\sum_{t = 1}^{T(R)}{\tau_{225}(D, t)},$$ where $D$ is the date we are looking at for the weather forecast, $R$ refers to the specific telescope, and $T(R)$ is the number of times forecasts being made in the telescope's observation timeframe. For each day, we calculate the day's reward by combining the telescopes' rewards in weighted average. $$F(D) = \sum_{i = 1}^N{W_{R_i}\times f(D, R_i)}.$$ Then we make decisions on whether to trigger the day based on the pure values of $F(D)$.

In the discounted (punishment for uncertainty) model, we calculate $$F(D, r) = {\sum_{i = 1}^N{W_{R_i}\times f(D, R_i)}}\times{(1+r)^D},$$ where $r$ is the punish level and then make decisions on $F(D,r)$. We multiply rather than divide here because the value is negative. We are going to experiment with different fixed value of $r$, and a function of $r$ depending on the variances in the forecasts, and compare the results with the ground-truth optimal path.

We also experimented with different weights. In the baseline model, we were using the output  file size as a measure of how important each telescope is. In the discounted model, we adjusted the weights by using the area of the telescope. We further scaled all $F(D, r)$ to be a weighted average of $f(D, R_i)$, by letting $W_{R_i}$ be the proportion of telescope $R_i$'s area in the total area. By doing this, we think the output can be more interpretable.

When we do evaluations, we are essentially looking backwards. Therefore, we directly calculate the scores without the discount factor $r$, for both the ground-truth path and our suggested path. The score given to any path $P$ is going to be $$S(P) = \frac{2}{1 + e^{- \sum_{j=1}^{\text{Days}}{F(D_{P,j})}}},$$ where $D_{P,j}$ is the $j$-th day selected by the path $P$. We have $2$ in the numerator, because the sigmoid function will give us values from 0 to 0.5 based on our negative $F$ values and we want to get scores from 0 to 1.


## Import Data

In [2]:
telescopes = ['12-meter','alma','apex','aste','iram','jcmt','lmt','sma','smt','spt']

In [3]:
starttime = datetime(2019,10,3,6)
endtime = datetime(2019,10,14,0) # not included
timestamps = np.arange(starttime, endtime, 
                       timedelta(hours=6)).astype(datetime)
databook = {}
for ts in telescopes:
    databook[ts] = dict.fromkeys(timestamps)

In [4]:
# for ts in telescopes:
#     for t in timestamps:
#         filepath = "data/"+ ts +"/"+ t.strftime("%Y%m%d_%H:%M:%S")
#         try:
#             df = pd.read_csv(filepath, delim_whitespace=True, skiprows = 1, header = None)
#             df.columns = ["date", "tau225", "Tb[k]", "pwv[mm]", "lwp[kg*m^-2]","iwp[kg*m^-2]","o3[DU]"]
#             df['date'] = pd.to_datetime(df['date'], format = "%Y%m%d_%H:%M:%S")
#             databook[ts][t] = df
#         except FileNotFoundError:
#             databook[ts][t] = None
# # databook is a dictionary of dictionaries of dataframes 
# # keys: telescope names
# # values: dictionaries of dataframes for one telescope
# # databook[telescope_name] is a dictionary of dataframes for one telescope
# # keys: timestamps when the forecast is made
# # values: forecast dataframe (None if missing)

In [5]:
################## fake data ###################
actual_time_span = np.arange(datetime(2019,7,1,0), datetime(2019,10,22,6), 
                       timedelta(hours=6)).astype(datetime)

i = 0
for site in telescopes:
    for t in timestamps:
        actual_time = actual_time_span[i]
        time_delta = t - actual_time
        filepath = "data/MaunaKea/"+ actual_time.strftime("%Y%m%d_%H:%M:%S")
        df = pd.read_csv(filepath, delim_whitespace=True, skiprows = 1, header = None)
        df.columns = ["date", "tau225", "Tb[k]", "pwv[mm]", "lwp[kg*m^-2]","iwp[kg*m^-2]","o3[DU]"]
        df['date'] = pd.to_datetime(df['date'], format = "%Y%m%d_%H:%M:%S") + time_delta
        databook[site][t] = df
        i += 1

In [6]:
print(telescopes[0],timestamps[1])
(databook[telescopes[0]][timestamps[1]]).head()

12-meter 2019-10-03 12:00:00


,date,tau225,Tb[k],pwv[mm],lwp[kg*m^-2],iwp[kg*m^-2],o3[DU]
0,2019-10-03 12:00:00,0.14078,39.414,3.2865,0.0,0.0,264.41
1,2019-10-03 13:00:00,0.14125,39.575,3.2760,0.0,0.0,264.04
2,2019-10-03 14:00:00,0.14093,39.556,3.2474,0.0,0.0,265.37
3,2019-10-03 15:00:00,0.13977,39.343,3.2021,0.0,0.0,265.87
4,2019-10-03 16:00:00,0.13965,39.350,3.1803,0.0,0.0,266.10


## Compute Variance

core: 
- the larger the punishment level is, the larger the difference between smaller and larger variance (in terms of ratio)
- when punishment_level = 0, all effect of variances should be 1.
- variance can be both <1 and >1, so $\text{variance}^{\text{punish_level}}$ does not work.

So far we use:
$$e^{\text{variance} * \text{punish_level}}$$

1. Variance v.s. date of prediction

2. Variance v.s. number of days forward (ideally using historical data to prevent information leak)

In [7]:
# columns are the timestamps when predictions were made, indices are timestamps that're being predicted
std_dict = {}
for site in telescopes:
    data_telescope = databook[site]
    df_tau225 = pd.concat([data_telescope[t].set_index('date')['tau225'] for t in data_telescope if data_telescope[t] is not None],axis = 1)
    df_tau225.columns = [t for t in data_telescope if data_telescope[t] is not None]

    # use df_mask to record the number of days forward (index - col) for each prediction
    df_mask = pd.DataFrame(index = df_tau225.index, columns = df_tau225.columns)
    for col in df_tau225.columns:
        df_mask[col] = (df_tau225.index - col).days

    # get 'true' tau225 value, which is the closest prediction within a day
    true_tau225 = pd.Series(index = df_tau225.index)
    for idx in df_tau225.index:
        values = df_tau225.loc[idx]
        true_tau225[idx] = values.values[~values.isna()][-1]

    # compute standard error v.s. # days forward
    df_diff = (df_tau225.T - true_tau225).T
    std = []
    for i in range(1, df_mask.max().max()+1):
        std.append((np.nanmean(df_diff[df_mask==i].values ** 2))**(1/2))
    
    std_dict[site] = std[:10]

## Define Model

In [8]:
def day_reward(telescope_name, day_current_str, end_day_str, start_time, end_time, \
            use_as_evaluate = False, time_variance = False, furthure_variance = False, punish_level = 1):
    '''
    For the specified telescope, return a dataframe with two columns.
    The first column tells the day in the day window between 
        day_current_str and end_day_str (inclusive).
    The second column tells the average predicted tao225 given the day and the time window between
        start_time and end_time (inclusive).
    
    '''
    split_day_current = day_current_str.split('-')
    split_day_end = end_day_str.split('-') # include this day
    
    day_current = datetime(int(split_day_current[0]),int(split_day_current[1]),int(split_day_current[2]),0)
    day_end = datetime(int(split_day_end[0]),int(split_day_end[1]),int(split_day_end[2])+1,0)
    
    if not use_as_evaluate:
        mask = [t < day_current for t in databook[telescope_name]]
        t_valid = np.array([t for t in databook[telescope_name]])[mask]

        df_all = pd.concat([databook[telescope_name][t] for t in t_valid], axis =0)
    else:
        df_all = pd.concat([databook[telescope_name][t] for t in databook[telescope_name]], axis =0)
    
    # day and time filter
    df_all = df_all[(df_all.date >= day_current) & (df_all.date < day_end)]
    df_all['day'] = df_all.date.apply(lambda x: str(x).split(' ')[0])
    df_all['time'] = df_all.date.apply(lambda x: int(str(x).split(' ')[1][0:2]))
    df_all = df_all[(df_all.time >= int(start_time)) & (df_all.time <= int(end_time))]
    
    # calculate the day reward 
    df_tau_all = df_all.groupby('date').agg({'tau225':lambda x: list(x)}).reset_index()
    df_tau_all['day'] = df_tau_all.date.apply(lambda x: str(x).split(' ')[0])
    
    df_tau_all['latest'] = df_tau_all['tau225'].apply(lambda x: x[-1]) 
    df_tau_all['mean'] = df_tau_all['tau225'].apply(lambda x: - np.mean(x)) 
    df_tau_all['std'] = df_tau_all['tau225'].apply(lambda x: np.std(x))
    
    if time_variance:
        df_tau_day = pd.DataFrame(df_tau_all.groupby('day').apply(lambda x: np.mean(x['mean'] * np.exp(punish_level * x['std']))))
        df_tau_day.columns = ['value']
    
    else:
        df_tau_day = pd.DataFrame(-df_tau_all.groupby('day')['latest'].mean())
        df_tau_day.columns = ['value']

        if furthure_variance:
            df_tau_day['value'] = df_tau_day['value'].values * np.exp(np.array(std_dict[telescope_name][:len(df_tau_day)]) * punish_level)  
        
    return df_tau_day

**Weighted sum the reward for each telescope according to the total Gbytes.** 
( still need the suggested schedule for '12-meter','aste','iram')

In [9]:
# weight_telescope = [0, 22830.7, 26153.8, 0, 0, 12123.0, 22215.3, 12123.0, 18030.7, 26953.8]

# using the area (radius ** 2) of the telescope as weights 
weight_telescope = [12**2, 73**2, 12**2, 10**2, 30**2, 15**2, 32.5**2, 14.7**2, 10**2, 6**2]
# schedule_telescope = [[0,1], [3,13], [3,15], [0,1], [0,1], [10,16], [6,16], [10,16], [8,16], [3,15]]
schedule_telescope = [[0,23], [3,13], [3,15], [0,23], [0,23], [10,16], [6,16], [10,16], [8,16], [3,15]]


dict_schedule = dict(zip(telescopes, schedule_telescope))
dict_weight = dict(zip(telescopes, weight_telescope))

In [10]:
def all_day_reward(day_current_str, end_day_str, time_variance = False, furthure_variance = False, punish_level = 1):
    """
    calculate F(D) for D in range(day_current_str, end_day_str)
    taking in every single telescope we currently have
    weighted their f reward values
    based on area_i/total_area
    """
    # set up a dataframe
    telescopes_day_reward = day_reward(telescopes[0], day_current_str, end_day_str, \
                                       dict_schedule[telescopes[0]][0], dict_schedule[telescopes[0]][1], \
                                       time_variance = time_variance, furthure_variance = furthure_variance, punish_level = punish_level) \
                                       * dict_weight[telescopes[0]] 
    # 
    for i in telescopes[1:]:
        telescopes_day_reward += day_reward(i, day_current_str, end_day_str, \
                                            dict_schedule[i][0], dict_schedule[i][1], \
                                            time_variance = time_variance, furthure_variance = furthure_variance,  punish_level = punish_level)\
                                            * dict_weight[i] 
    return telescopes_day_reward / sum(weight_telescope)

### Fixed Punish Level: Making Suggestions On-the-Go

In [11]:
def decision_making_single_punishment(day_current_str, end_day_str, days_to_trigger, punish_level = 0):
    # day_current_str: YYYY-MM-DD (str) (included)
    # end_day_str: YYYY-MM-DD (str) (included)
    # days_to_trigger: days to trigger (int)
    each_day_reward = all_day_reward(day_current_str, end_day_str)
    
    # inflate the values on each day
    a = np.array([n * ((1 + punish_level) ** i) for i, n in enumerate(each_day_reward['value'])])
    
    # select the 'days_to_trigger' number of days having maximum reward values
    selected_days = np.array(each_day_reward.index)[np.argsort(a)[-1:-days_to_trigger-1:-1]]
    if day_current_str in selected_days:
        print('We suggest triggering on today {}'.format(day_current_str))
        output = True
    else: 
        print('We DO NOT suggest triggering on today {}'.format(day_current_str))
        output = False
    print('And we suggest to trigger by the following sequence: {}'.format(np.array(sorted(selected_days))))
    dic = dict(zip(each_day_reward.index, a))
    print('The discounted reward values for all the future days are ', dic)
    return output
          

### Punish Level According To Variance (how far it is predicting)

In [12]:
def decision_making_further_variance_punishment(day_current_str, end_day_str, days_to_trigger, punish_level = 0):
    # day_current_str: YYYY-MM-DD (str) (included)
    # end_day_str: YYYY-MM-DD (str) (included)
    # days_to_trigger: days to trigger (int)
    each_day_reward = all_day_reward(day_current_str, end_day_str, furthure_variance = True, punish_level = punish_level)

    # select the 'days_to_trigger' number of days having maximum reward values
    selected_days = each_day_reward.sort_values(by='value', ascending=False)[:days_to_trigger].index
    
    if day_current_str in selected_days:
        print('We suggest triggering on today {}'.format(day_current_str))
        output = True
    else: 
        print('We DO NOT suggest triggering on today {}'.format(day_current_str))
        output = False
    print('And we suggest to trigger by the following sequence: {}'.format(np.array(sorted(selected_days))))
    dic = each_day_reward.to_dict()['value']
    print('The discounted reward values for all the future days are ', dic)
    return output

### Punish Level According To Variance (the predictions for a specific time)

In [13]:
def decision_making_time_variance_punishment(day_current_str, end_day_str, days_to_trigger, punish_level = 0):
    # day_current_str: YYYY-MM-DD (str) (included)
    # end_day_str: YYYY-MM-DD (str) (included)
    # days_to_trigger: days to trigger (int)
    each_day_reward = all_day_reward(day_current_str, end_day_str, time_variance = True, punish_level = punish_level)

    # select the 'days_to_trigger' number of days having maximum reward values
    selected_days = each_day_reward.sort_values(by='value', ascending=False)[:days_to_trigger].index
    
    if day_current_str in selected_days:
        print('We suggest triggering on today {}'.format(day_current_str))
        output = True
    else: 
        print('We DO NOT suggest triggering on today {}'.format(day_current_str))
        output = False
    print('And we suggest to trigger by the following sequence: {}'.format(np.array(sorted(selected_days))))
    dic = each_day_reward.to_dict()['value']
    print('The discounted reward values for all the future days are ', dic)
    return output

## Evaluations

In [14]:
def simulate_process(days, num_days_trigger, function, punish_level):
    decisions = []
    for curr_day in days:
        days_left = int(num_days_trigger - np.sum(decisions))
        if days_left == 0:
            pass
        else:
            decisions.append(\
                           function(curr_day, days[-1], days_left, punish_level))
        print("")
    
    suggested_path = days[decisions + [False] * (len(days) - len(decisions))]
    return suggested_path


In [15]:
def best_path_afterwards(start_day_str, end_day_str, days_to_trigger, exp_conversion, days_have_triggered = None):
    # start_day_str: YYYY-MM-DD (str) (included)
    # end_day_str: YYYY-MM-DD (str) (included)
    # days_to_trigger: days to trigger (int)
    # exp_conversion: function that map the negative value to [0, 1]
    # days_have_triggered: days acutally triggered (list of str)
    telescopes_day_reward = day_reward(telescopes[0], start_day_str, end_day_str, dict_schedule[telescopes[0]][0], dict_schedule[telescopes[0]][1], use_as_evaluate=True) * dict_weight[telescopes[0]]
    for i in telescopes[1:]:
        telescopes_day_reward += day_reward(i, start_day_str, end_day_str, dict_schedule[i][0], dict_schedule[i][1], use_as_evaluate=True) * dict_weight[i]
    telescopes_day_reward = telescopes_day_reward / sum(weight_telescope)
    
    all_path = telescopes_day_reward.sort_values(by='value', ascending = False)
    best_path = all_path[:days_to_trigger]
    random_path = np.random.choice(all_path.index, days_to_trigger, replace = False)
    print('The suggested path we predicted on the go is      {}'\
         .format(days_have_triggered))    
    print('The best path to trigger based on ground-truth is {}'\
          .format(np.array(sorted(best_path.index))))
    print('The random path to trigger is                     {}'\
          .format(np.array(sorted(random_path))))

    if days_have_triggered is not None:
        print('\nThe score given to the suggested path is          {}'\
              .format(exp_conversion(all_path.loc[days_have_triggered]['value'])))
    print('The score given to the best path is               {}'\
          .format(exp_conversion(best_path['value'])))
    print('The score given to the random path is             {}'\
          .format(exp_conversion(all_path.loc[random_path]['value'])))

    return all_path

In [16]:
whole_time_window = all_day_reward('2019-10-05', '2019-10-14')
days = np.array(whole_time_window.index)
suggested_path = simulate_process(days, 5, decision_making_single_punishment, 0.1)

We suggest triggering on today 2019-10-05
And we suggest to trigger by the following sequence: ['2019-10-05' '2019-10-06' '2019-10-07' '2019-10-08' '2019-10-13']
The discounted reward values for all the future days are  {'2019-10-05': -0.1444145363905571, '2019-10-06': -0.12534579854967196, '2019-10-07': -0.17935825364206198, '2019-10-08': -0.16590859152256887, '2019-10-09': -0.35755155652695064, '2019-10-10': -0.32807848389134914, '2019-10-11': -0.3338484696338396, '2019-10-12': -0.3445681334921599, '2019-10-13': -0.227712445596289, '2019-10-14': -0.24744430748626783}

We suggest triggering on today 2019-10-06
And we suggest to trigger by the following sequence: ['2019-10-06' '2019-10-07' '2019-10-08' '2019-10-09']
The discounted reward values for all the future days are  {'2019-10-06': -0.13931276546644886, '2019-10-07': -0.14624908553479665, '2019-10-08': -0.15100488032522855, '2019-10-09': -0.17712320866378195, '2019-10-10': -0.27047643621685113, '2019-10-11': -0.40737569067188784,

In [17]:
whole_time_window = all_day_reward('2019-10-05', '2019-10-14')
days = np.array(whole_time_window.index)
suggested_path = simulate_process(days, 5, decision_making_further_variance_punishment, 0.1)

We suggest triggering on today 2019-10-05
And we suggest to trigger by the following sequence: ['2019-10-05' '2019-10-06' '2019-10-08' '2019-10-13' '2019-10-14']
The discounted reward values for all the future days are  {'2019-10-05': -0.14693705855562592, '2019-10-06': -0.11732646609949775, '2019-10-07': -0.15206425273199173, '2019-10-08': -0.12780026495811003, '2019-10-09': -0.25260675117528486, '2019-10-10': -0.20907876739941544, '2019-10-11': -0.19326507895314232, '2019-10-12': -0.18210285545953298, '2019-10-13': -0.10973242972044575, '2019-10-14': -0.10833626978808757}

We DO NOT suggest triggering on today 2019-10-06
And we suggest to trigger by the following sequence: ['2019-10-07' '2019-10-08' '2019-10-09' '2019-10-13']
The discounted reward values for all the future days are  {'2019-10-06': -0.1415951828762349, '2019-10-07': -0.1369674705337123, '2019-10-08': -0.12793406274440994, '2019-10-09': -0.1368887956479974, '2019-10-10': -0.1898414263287113, '2019-10-11': -0.2596158827

In [18]:
whole_time_window = all_day_reward('2019-10-05', '2019-10-14')
days = np.array(whole_time_window.index)
suggested_path = simulate_process(days, 5, decision_making_time_variance_punishment, 0.1)

We suggest triggering on today 2019-10-05
And we suggest to trigger by the following sequence: ['2019-10-05' '2019-10-06' '2019-10-07' '2019-10-08' '2019-10-11']
The discounted reward values for all the future days are  {'2019-10-05': -0.13243849904403848, '2019-10-06': -0.12161995065761698, '2019-10-07': -0.16921350192269902, '2019-10-08': -0.16373792909290136, '2019-10-09': -0.2791885738557433, '2019-10-10': -0.22126367695605662, '2019-10-11': -0.18606743009489204, '2019-10-12': -0.19107653737175037, '2019-10-13': -0.18698780188877803, '2019-10-14': -0.19071616842195252}

We suggest triggering on today 2019-10-06
And we suggest to trigger by the following sequence: ['2019-10-06' '2019-10-07' '2019-10-08' '2019-10-13']
The discounted reward values for all the future days are  {'2019-10-06': -0.12481163794562951, '2019-10-07': -0.16052099552991256, '2019-10-08': -0.1496183405254709, '2019-10-09': -0.21736731570394638, '2019-10-10': -0.19994000654901548, '2019-10-11': -0.200003604023098

In [19]:
exp_function = lambda x: 2 / (1 + np.exp(- x.sum()))

In [20]:
np.random.seed(2019)
all_state = best_path_afterwards('2019-10-05', '2019-10-14', 5, exp_function, days_have_triggered = suggested_path)


The suggested path we predicted on the go is      ['2019-10-05' '2019-10-06' '2019-10-07' '2019-10-08' '2019-10-13']
The best path to trigger based on ground-truth is ['2019-10-06' '2019-10-07' '2019-10-08' '2019-10-09' '2019-10-12']
The random path to trigger is                     ['2019-10-06' '2019-10-07' '2019-10-12' '2019-10-13' '2019-10-14']

The score given to the suggested path is          0.6424974899928185
The score given to the best path is               0.6777753688078361
The score given to the random path is             0.5368666688988142
